In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model,tree,metrics
import numpy as np

Original Data


In [2]:
pharma_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_advc.csv')

In [3]:
pharma_data.head()

,ID_Patient_Care_Situation,Diagnosed_Condition,Patient_ID,Treated_with_drugs,Patient_Age,Patient_Body_Mass_Index,Patient_Smoker,Patient_Rural_Urban,Patient_mental_condition,A,B,C,D,E,F,Z,Number_of_prev_cond,Survived_1_year
0,16201,47,8433,DX2,60,21.655523,NO,URBAN,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1
1,9421,3,2972,DX6,2,28.852743,NO,RURAL,Stable,1.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,0
2,16205,7,8608,Dx6,20,26.179725,NO,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1
3,5582,31,10074,dx6,8,22.638945,NO,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0
4,20880,43,7462,dx1,53,21.326131,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


Dropping Duplicates



In [4]:
pharma_data.drop_duplicates(inplace = True)

DATA PRE-PROCESSING

In [5]:
pharma_data['Patient_Smoker'].replace(['NO '],'NO',inplace=True)
pharma_data['Patient_Smoker'].replace(['YESS ','YES ','YESS'],'YES',inplace=True)
pharma_data['Patient_Smoker'].replace(['Cannnot say ','Cannot say ','Cannot say','CANNOT SAY ','CANNOT SAY'],'NO',inplace=True)

In [6]:
pharma_data['Treated_with_drugs'] = pharma_data['Treated_with_drugs'].str.lower().str.strip()
mode = pharma_data['Treated_with_drugs'].mode()
pharma_data['Treated_with_drugs'].fillna(mode[0],inplace = True)
pharma_data1 = pharma_data['Treated_with_drugs'].str.get_dummies(sep = ' ')

In [7]:
pharma_data['A'].replace(np.nan,pharma_data['A'].median(),inplace = True)
pharma_data['B'].replace(np.nan,pharma_data['B'].median(),inplace = True)
pharma_data['C'].replace(np.nan,pharma_data['C'].median(),inplace = True)
pharma_data['D'].replace(np.nan,pharma_data['D'].median(),inplace = True)
pharma_data['E'].replace(np.nan,pharma_data['E'].median(),inplace = True)
pharma_data['F'].replace(np.nan,pharma_data['F'].median(),inplace = True)
pharma_data['Z'].replace(np.nan,pharma_data['Z'].median(),inplace = True)
pharma_data['Number_of_prev_cond'] = pharma_data['A'] + pharma_data['B']+ pharma_data['C']+pharma_data['D']+ pharma_data['E']+ pharma_data['F']+ pharma_data['Z']

In [8]:
pharma_data[pharma_data1.columns] = pharma_data1
pharma_data = pd.get_dummies(pharma_data,columns=['Patient_Smoker','Patient_Rural_Urban'])

In [9]:
pharma_data.drop(columns = ['Patient_mental_condition','Patient_ID','Treated_with_drugs'],axis = 1,inplace = True)

In [10]:
pharma_data.isnull().sum()

ID_Patient_Care_Situation    0
Diagnosed_Condition          0
Patient_Age                  0
Patient_Body_Mass_Index      0
A                            0
B                            0
C                            0
D                            0
E                            0
F                            0
Z                            0
Number_of_prev_cond          0
Survived_1_year              0
dx1                          0
dx2                          0
dx3                          0
dx4                          0
dx5                          0
dx6                          0
Patient_Smoker_NO            0
Patient_Smoker_YES           0
Patient_Rural_Urban_RURAL    0
Patient_Rural_Urban_URBAN    0
dtype: int64

In [11]:
X = pharma_data.drop(columns = ['Survived_1_year'])
y = pharma_data['Survived_1_year'	]

### **Splittnig Data in Train and Test**

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 4)

In [13]:
# !pip install lazypredict
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose = 0,ignore_warnings = True,custom_metric = None)
model,predictions = clf.fit(X_train,X_test,y_train,y_test)
model

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
100%|██████████| 30/30 [01:44<00:00,  3.48s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.80,0.78,0.78,0.80,0.47
XGBClassifier,0.79,0.76,0.76,0.79,1.20
RandomForestClassifier,0.78,0.75,0.75,0.78,2.26
AdaBoostClassifier,0.78,0.75,0.75,0.78,0.76
BaggingClassifier,0.76,0.75,0.75,0.76,0.68
ExtraTreesClassifier,0.75,0.72,0.72,0.75,1.87
NuSVC,0.74,0.71,0.71,0.74,22.68
SVC,0.74,0.71,0.71,0.74,14.82
DecisionTreeClassifier,0.72,0.70,0.70,0.72,0.14


### **Different Models for finding best f1_score**


In [14]:
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
BGGrf = BaggingClassifier(RandomForestClassifier(n_estimators = 120))
BGGrf.fit(X_train, y_train)
predictions=BGGrf.predict(X_test)
print(metrics.f1_score(y_test,predictions))

0.8607875762617858


In [16]:
from lightgbm.sklearn import LGBMClassifier
LGBMModel = LGBMClassifier()
LGBMModel.fit(X_train,y_train)
prediction = LGBMModel.predict(X_test)
print(metrics.f1_score(y_test,prediction))

0.8777614138438881


In [17]:
test_new = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_advc.csv')

In [18]:
test_new.head()

,ID_Patient_Care_Situation,Diagnosed_Condition,Patient_ID,Treated_with_drugs,Patient_Age,Patient_Body_Mass_Index,Patient_Smoker,Patient_Rural_Urban,Patient_mental_condition,A,B,C,D,E,F,Z,Number_of_prev_cond
0,24206,35,4640,DX5,65,20.71,NO,RURAL,Stable,1.00,0.00,0.00,0.00,1.00,0.00,0.00,2.00
1,32827,30,3214,dx1,2,24.25,NO,URBAN,Stable,1.00,0.00,0.00,0.00,1.00,0.00,0.00,2.00
2,3694,46,3564,dx6,1,27.14,NO,URBAN,Stable,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,7164,44,5176,DX1,29,29.19,NO,RURAL,Stable,0.00,0.00,1.00,0.00,1.00,0.00,0.00,2.00
4,1259,30,1101,DX5,51,20.84,NO,URBAN,Stable,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00


In [19]:
test_new['Treated_with_drugs'] = test_new['Treated_with_drugs'].str.lower().str.strip()
pharma_data1 = test_new['Treated_with_drugs'].str.get_dummies(sep = ' ')
test_new[pharma_data1.columns] = pharma_data1

In [20]:
test_new['Patient_Smoker'].replace(['NO '],'NO',inplace = True)
test_new['Patient_Smoker'].replace(['YESS','YESS ','YES '],'YES',inplace = True)

In [21]:
test_new = pd.get_dummies(test_new,columns=['Patient_Smoker','Patient_Rural_Urban'])
test_new.drop(columns = ['Patient_mental_condition','Patient_ID','Treated_with_drugs'],axis = 1,inplace = True)


## **Predicting through LGBMClassifier**

In [22]:
prediction = LGBMModel.predict(test_new)

In [23]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(prediction)
res.index = test_new.index # its important for comparison
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results.csv')         
files.download('prediction_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>